# Text locations to map

This takes a dataframe, extracts all place names from text column, gets counts of these and then collects coordnates for them.

It then maps these on a map. 

A category column(s) is also included, for mapping two different categories.

NEXT TO DO:

Split this code into 1) dataframe prep and coordinate retrival and 2) actual mapping

In [97]:
# Check this
#https://pyrosm.readthedocs.io/en/latest/basics.html

In [98]:
# Usual imports

import pandas as pd
import codecs

from collections import Counter

from itertools import chain

from os import listdir
from os.path import isfile, join

In [99]:
# Map imports

import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point

In [100]:
PATH = './DATA/'

In [ ]:
# Get dataframes available

onlyfiles = [f for f in listdir(PATH) if isfile(join(PATH, f))]
print(onlyfiles)

In [102]:
# Which file to use

filename = onlyfiles[0]

In [ ]:
# Load dataframe

CONVERTERS = {'pos_tokens': eval,
            'jkeywords_clean': eval}

df = pd.read_csv(PATH + filename, converters=CONVERTERS, lineterminator='\n')

In [ ]:
# Checking

df.head(1)

# Specific POS extraction

In [8]:
# Getting the two positions of speech I need for locations/places

def pos_extract(pos, pos2, txt):
      x = [token for token in txt if token.endswith(pos) or token.endswith(pos2)]
      y = [token.split('/')[0] for token in x] # use when I need lists with just these!
      #return len(x)
      return y

In [9]:
# In my chinese data/jieba output = ns and nrt mostly places? #nr = people ?

df['place_tags'] = df['pos_tokens'].apply(lambda x: pos_extract('nrt', 'ns', x)) 

# Get list and count of locations

In [14]:
# Making list of place names and how often they appear

count1 = pd.Series(Counter(chain.from_iterable(df['place_tags'])))
count1 = count1.sort_values(ascending=False)

In [16]:
count1.to_csv(PATH + 'places.csv', index=True)

# Re-Start here for collection

In [104]:
df = pd.read_csv(PATH + 'places.csv', names=['place', 'n'], header=0, index_col=None)

In [ ]:
## Old code - use if place name and POS are extracted

#df[['city', 'pos']] = df['city'].str.split('/', 1, expand=True)

# Geopy 

This works, but not integrated in task.
This returns no errors for non-place names, so would need a really better filter for these or another means to detect them
Ideal world would be nice - smoother than Selenium calls

In [125]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [123]:
# For testing
df_slice = df[:100] 

In [ ]:
# Some other options
# from functools import partial


# geolocator = Nominatim(user_agent="specify_your_app_name_here")
# geocode = partial(geolocator.geocode, language="es")

# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# df['location'] = df['name'].apply(geocode)
# df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [126]:
lat_list = []
long_list = []

loc_list = []
count = []

In [ ]:
for index, row in df_slice.iterrows():

    geolocator = Nominatim(user_agent="Test")

    location = geolocator.geocode(row['place'])
    print(location)

    loc_list.append(row['place'])
    lat_list.append(location.latitude)
    long_list.append(location.longitude)
    count.append(row['n']) #n_count

    time.sleep(2)
    

In [130]:
geopy_map_df = pd.DataFrame(zip(loc_list, lat_list, long_list, count))

In [131]:
geopy_map_df = geopy_map_df.rename(columns={0: 'location', 1: 'latitude',
            2: 'longitude',
            3: 'n_count'})

In [ ]:
geopy_map_df

In [ ]:
# Save here

# Collect coordinates - Selenium and Google

In [122]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service

import time


In [22]:
coord_list = []
loc_list = []
key_list = []
count = []

In [47]:
# For testing
df_slice = df[:1000] 

In [90]:
# Where is Chromedriver
# Version 129.0.6668.70

driver_location = './INPUT/chromedriver' 

In [ ]:
options = webdriver.ChromeOptions()

# First seems super needed on my system
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
options.add_argument("--disable-infobars")
#options.add_argument("--disable-extensions")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

options.add_argument("user-data-dir=selenium")

#options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(driver_location), options=options)

for index, row in df_slice.iterrows():
      keyword = str(row['place']) #name
      #url_search = 'https://www.google.com/maps/search/?api=1&query=' + lat + ', ' + long
      url_search = 'https://www.google.com/maps/'

      driver.get(url_search)

      driver.implicitly_wait(1000)

      ActionChains(driver).move_to_element(test).click().send_keys(keyword).perform() # 社会信用体系

      test.send_keys(Keys.RETURN)

      driver.implicitly_wait(1000)

      while driver.current_url == 'https://www.google.com/maps/':
            driver.implicitly_wait(10)
      
      time.sleep(3)
      driver.implicitly_wait(500)

      url = driver.current_url


      core, api = url.split('@')

      try:
            coordinates = api.split(',')[:2]
            print(coordinates)
            coord_list.append(coordinates)
            key_list.append(keyword)
            count.append(row['n']) #n_count
      except:
            print('Coordinate parsing failed')
      
      try:
            location = core.split('place')[-1].replace('/', '')
            print(location)
            loc_list.append(location)
      
      except:
            print('Location parsing failed')

      try:
            with codecs.open(PATH + 'running_geo_data.csv', 'a', 'utf-8',) as f:
                  f.write(str(coordinates) + ';' + location + ';' + keyword + ';' + str(row['n']) + ';' + str(coordinates[0]) + ';' +  str(coordinates[1]) + '\n')
      except:
           with codecs.open(PATH + 'running_geo_data.csv', 'a', 'utf-8',) as f:
                 f.write(keyword + ':' + 'NONE' + ';' + 'NONE' + ';' + str(row['n']) + '\n')

      print(index)

      time.sleep(4)
      # only needed to merge from time to time

driver.close()


In [95]:
#driver.close()

# Dataframe work

Though running data was saved, making dataframe here from running lists

In [96]:
map_df = pd.DataFrame(zip(coord_list, loc_list, key_list, count))

In [ ]:
map_df

In [ ]:
map_df = map_df.rename(columns={0: 'coordinates', 1: 'location',
            2: 'name',
            3: 'n_count'})

In [ ]:
temp = pd.DataFrame(map_df['coordinates'].to_list(), columns=['latitude', 'longitude'])
map_df = pd.concat([map_df, temp], axis=1)

In [ ]:
map_df_update = pd.concat([map_df, keep])
map_df_update.reset_index(inplace=True, drop=True)
map_df_update.sort_values(by='n_count', axis=0)

In [ ]:
map_df.to_csv(PATH + 'map_coords.csv', index=False)

# Catategory (dummy variable) work to add a mapping option

Will come back to clean this- basic task, not interesting

In [ ]:
temp_df = df[['place_tags', 'category']]
temp_df = temp_df[temp_df.place_tags.map(len) > 0]

#df[df['amp'].map(len) == 495]

In [ ]:
temp_df.category.value_counts()

In [ ]:
temp_df.head(2)

In [ ]:
CONVERTERS = {'coordinates': eval}
map_df = pd.read_csv(PATH + 'map_coords.csv', converters=CONVERTERS)

In [ ]:
map_df.head(2)

In [ ]:
# IS this old now? Not the way I prefer to do it

def get_categories_rev(x):
      p = []

      for index, row in map_df.iterrows():
            if row['name'] in x:
                  print(row['name'])
                  p.append(row.coordinates)
                  #return(row.category)
            else:
                  pass
      #print(p)
      return p

In [ ]:
test = get_categories_rev(temp_df['place_tags'][0])

In [ ]:
len(test)

In [ ]:
temp_df['coordinates'] = temp_df['place_tags'].apply(lambda x: get_categories_rev(x))

In [ ]:
new_dict = {'test': []}

In [ ]:
# Faster now by about 33%

def get_categories(x):
      new_dict = {}
      for index, row in temp_df.iterrows():
            if x in row.place_tags:
                  if x not in list(new_dict.keys()):
                        new_dict[x] = []
                  new_dict[x].append(row.category)
                  #p.append(row.category)
                  #return(row.category)
            else:
                  pass
            # try:
            #       print(new_dict[x])
            # except:
            #       pass
      print(len(new_dict[x]))
      return new_dict[x]

In [ ]:
map_df['categories'] = map_df['name'].apply(lambda x: get_categories(x))


In [ ]:
map_df.to_csv(PATH + 'map_coords.csv', index=False)

# Import and DF work

In [8]:
CONVERTERS = {'latitude': float, 'longitude': float}

map_df = pd.read_csv(PATH + 'map_coords.csv', converters=CONVERTERS)


In [ ]:
map_df.head(10)

In [ ]:
## Only needed to merge from time to time

# map_df_2 = pd.read_csv('../data/map_coords_final_DEL.csv', converters=CONVERTERS)
# map_df = pd.concat([map_df_1, map_df_2])
# map_df.reset_index(inplace=True, drop=True)
# len(map_df)

# Categories for map work

Here we get counts for the categories for each location

NOTE - move to first catagory section and save this, and delete old catagories column as that list is not needed!

In [21]:
CONVERTERS = {'latitude': float, 'longitude': float, 'categories':eval}

map_df = pd.read_csv(PATH + 'map_coords.csv', converters=CONVERTERS)

In [ ]:
map_df.columns

In [23]:
def row_count(x):
      count = pd.Series(Counter(x))
      count_dict = count.to_dict()
      return count_dict

In [24]:
map_df['cat_dict'] = map_df['categories'].apply(lambda x: row_count(x)) 

In [25]:
df = pd.json_normalize(map_df['cat_dict'])

In [26]:
map_df = pd.merge(map_df, df, left_index=True, right_index=True)


In [ ]:
map_df.head(2)

#### This all relates to bad locations, i.e. which return where you are.

Would like to find a neater way to manage this, including also filtering before we collect coordinates (to avoid uneeded requests)

In [ ]:
len('https:www.google.commapssearch%E6%B7%B1%E5%BA%A6')

In [29]:
# Not sure why I had a recollect here

recollect = map_df[map_df['coordinates'] == "['52.5225887', '13.2993244']"]
keep = map_df[map_df['coordinates'] != "['52.5225887', '13.2993244']"]

In [30]:
# very rough way to drop bad locations
map_df = map_df[map_df.location.str.len() < 45]

In [ ]:
len(map_df)

# Start visualization code

In [112]:
#map_df = pd.read_csv(PATH + 'map_coords.csv')

In [ ]:
map_df

In [ ]:
lat = []
long = []
n_count = []
s_count = [] # a category count
p_count = [] # a category count


for index, row in map_df.iterrows():
      if row.latitude > 60 or row.latitude < 15 or row.longitude > 135 or row.longitude < 70:
            print('Outside China')
            pass
      else:
            lat.append(row.latitude)
            long.append(row.longitude)
            n_count.append(row.n_count)
            s_count.append(row.scientific)
            p_count.append(row.political)                        
            print(row.n_count)
      
      #15-60 lat
      #70-135 lon

In [51]:
# If list is all in China or for other maps
#lat = map_df['latitude'].to_list()
#long = map_df['longitude'].to_list()

In [35]:
geometry = [Point(xy) for xy in zip(long,lat)]

### Maps digression

In [49]:
MAPS_PATH = '/home/jesselehrke/Documents/china_map_data/'

In [ ]:
# to process existing csv
mapfiles = [f for f in listdir(MAPS_PATH) if isfile(join(MAPS_PATH, f))]
print('Index, Filename' + '\n')
print(list(zip([str(index) for index, value in enumerate(mapfiles)], mapfiles)))

In [51]:
#map_file = 'china-latest.osm.pbf'
river_file = mapfiles[89]
map_file = mapfiles[93]
land_use = mapfiles[9]

#map_file = 'china-provinces.json'
#https://www.kaggle.com/datasets/gpreda/china-regions-map?resource=download

In [52]:
map_link = MAPS_PATH + map_file

# river_link = MAPS_PATH + river_file
# use_link = MAPS_PATH + land_use

In [ ]:
china = gpd.read_file(map_link, bbox=None, mask=None, rows=None)

# rivers = gpd.read_file(river_link, bbox=None, mask=None, rows=None)
# use = gpd.read_file(use_link, bbox=None, mask=None, rows=None)

In [ ]:
dir(gpd.plotting)

In [ ]:
fig,ax = plt.subplots(figsize = (20,7))
china.plot(ax = ax, facecolor="none",  linewidth=0.5, edgecolor='red', alpha=.4)
#rivers.plot(ax = ax, linewidth=0.2, alpha=.9)
# use.plot(ax = ax, linewidth=0.2, alpha=.9)

fig.suptitle('China')
ax.set_axis_off()

plt.show()

In [ ]:
china.loc[0, 'geometry']

### Back on task

In [67]:
geo_df = gpd.GeoDataFrame(geometry = geometry)

In [68]:
geo_df['n'] = n_count

In [69]:
geo_df['p'] = p_count
geo_df['s'] = s_count

In [70]:
geo_df = geo_df[1:]

In [61]:
#geo_df.to_csv('../data/map_coords_for_mapping.csv', index=False)

In [62]:
#geo_df = pd.read_csv('../data/map_coords_for_mapping.csv')

In [ ]:
geo_df.head(1)

In [72]:
p1 = []
s1 = []
for index, row in geo_df.iterrows():
      if row.p > row.s:
            p1.append(row.p * 100 / row.s)
            s1.append(0)
      else:
            s1.append(row.s * 100 / row.p)
            p1.append(0)


In [73]:
geo_df['p1'] = p1
geo_df['s1'] = s1

In [ ]:
geo_df.head(4)

In [ ]:
fig,ax = plt.subplots(figsize = (30,30))
china.plot(ax = ax)

g = geo_df.plot(ax = ax, markersize = geo_df['p1'], color = 'orange',marker = '.',label = 'Locations', alpha=.2)
g = geo_df.plot(ax = ax, markersize = geo_df['s1'], color = 'green',marker = '.',label = 'Locations', alpha=.9)

plt.savefig("./OUTPUTS/map_test.png", format='png', dpi=150, bbox_inches='tight')

plt.show()

#### Not sure what I was doing here on

In [ ]:
data_merged = gpd.sjoin(china, geo_df, how="inner")# op='within')

In [ ]:
data_merged.columns

In [ ]:
# Where did name_1 come from ?

data_merged.groupby('name_1').agg('sum')

In [ ]:
# Not working as name_1 not specified

unit = geo_df.dissolve(by='name_1', aggfunc='sum')

fig,ax = plt.subplots(figsize = (30,30))
china.plot(ax = ax)

g = unit.plot(ax = ax, markersize = geo_df['n'], color = 'red',marker = '.',label = 'Locations', alpha=.2)

plt.savefig("./OUTPUTS/map_total.png", format='png', dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
fig,ax = plt.subplots(figsize = (30,30))
china.plot(ax = ax)

g = geo_df.plot(ax = ax, markersize = geo_df['n'], color = 'red',marker = '.',label = 'Locations', alpha=.2)
plt.savefig("./OUTPUTS/map_total.png", format='png', dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
fig,ax = plt.subplots(figsize = (30,30))
china.plot(ax = ax)

#g = geo_df.plot(ax = ax, markersize = geo_df['s'], color = 'orange',marker = '.',label = 'Locations', alpha=.2)
g = geo_df.plot(ax = ax, markersize = geo_df['p'], color = 'red',marker = '.',label = 'Locations', alpha=.2)

plt.savefig("./OUTPUTS/map_pol.png", format='png', dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
fig,ax = plt.subplots(figsize = (30,30))
china.plot(ax = ax)

g = geo_df.plot(ax = ax, markersize = geo_df['s'], color = 'red',marker = '+',label = 'Locations', alpha=.4)
g = geo_df.plot(ax = ax, markersize = geo_df['p'], color = 'green',marker = 'x',label = 'Locations', alpha=.4)

plt.savefig("./OUTPUTS/map_sci.png", format='png', dpi=150, bbox_inches='tight')

plt.show()

In [95]:
# Must just be some reference code? 

# ward.crs = {'init':"epsg:4326"}
# geo_df.crs = {'init':"epsg:4326"}

# # plot the polygon
# ax = ward.plot(alpha=0.35, color='#d66058', zorder=1)
# # plot the boundary only (without fill), just uncomment
# #ax = gpd.GeoSeries(ward.to_crs(epsg=3857)['geometry'].unary_union).boundary.plot(ax=ax, alpha=0.5, color="#ed2518",zorder=2)
# ax = gpd.GeoSeries(ward['geometry'].unary_union).boundary.plot(ax=ax, alpha=0.5, color="#ed2518",zorder=2)

# # plot the marker
# ax = geo_df.plot(ax = ax, markersize = 20, color = 'red',marker = '*',label = 'Delhi', zorder=3)

# ctx.add_basemap(ax, crs=geo_df.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
# plt.show()